# Suscripción de Azure Quantum
### Conexión al espacio de trabajo de Azure Quantum

In [2]:
%azure.connect "/subscriptions/b3ed2c3e-e784-4a4e-9474-a3d279d2f9e4/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/LaboratorioQuantum-Ximena-Toledo-RIvera" location="eastus"


Authenticated using Microsoft.Azure.Quantum.Authentication.TokenFileCredential


Connected to Azure Quantum workspace LaboratorioQuantum-Ximena-Toledo-RIvera in location eastus.


Target ID,Current Availability,Average Queue Time (Seconds)
microsoft.estimator,Available,0
rigetti.sim.qvm,Available,5
rigetti.qpu.aspen-m-2,Available,5
rigetti.qpu.aspen-m-3,Available,5


# Implementación del Algoritmo de Shor
## Fase 1 
1. Invocación de bibliotecas necesarias.
2. Creación de un generador de números pseudoaleatorios.
    + Se inicializa un bit cuántico.
    + Se coloca en superposición.
    + Se somete al proceso de medición.
    + Se retorna el resultado.


In [3]:
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Oracles;
open Microsoft.Quantum.Random;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Characterization;

operation ramdomNumberGeneratorTest() : Result {// Complejidad algorítmica: O(10) = O(1)   
        use q1 = Qubit();                       // O(1)
        Message("Estado inicial de Qubit:");    // O(1)
        DumpMachine();                          // O(1)
        H(q1);                                  // O(1)
        Message("Estado de superpocision:");    // O(1)
        DumpMachine();                          // O(1)
        let measuredQubit = M(q1);              // O(1)
        Message("Proceso de medicion:");        // O(1)
        DumpMachine();                          // O(1)
        return measuredQubit;                   // O(1)
    }


ramdomNumberGeneratorTest

In [3]:
%simulate ramdomNumberGeneratorTest

Estado inicial de Qubit:


|0⟩	1 + 0𝑖
|1⟩	0 + 0𝑖

Estado de superpocision:


|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0.7071067811865476 + 0𝑖

Proceso de medicion:


|0⟩	0 + 0𝑖
|1⟩	1 + 0𝑖

One

In [4]:
operation ramdomNumberGenerator() : Result { 
                                        // Complejidad algorítmica: O(4) = O(1)
        use q1 = Qubit();               // O(1)
        H(q1);                          // O(1) // ?
        let measuredQubit = M(q1);      // O(1)
        return measuredQubit;           // O(1)
    }

ramdomNumberGenerator

## Generador de número pseudoaleatorio
1. Definimos un número máximo y mínimo
2. Definición de la cantidad de bits cuánticos expresados en valores enteros para llegar al valor máximo.
3. Crear cadena de longitud n.
4. Si la cadena de bits supera el valor máximo, crear cadena de longitud n.
5. Si la cadena de bits es menor al valor mínimo, crear cadena de longitd n.
6. Retornar valor aleatorio.

In [5]:
operation randomNumberRange(min: Int, max: Int) : Int {
                                                                // Complejidad algorítmica: O(8) = O(1), O(n) * O(2n) = O(n²) 
        mutable output = 0;                                     // O(1)
        repeat {                                                // O(n)
            // Bucle para generar números aleatorios hasta que genere uno que sea igual o menor que el máximo o igual o mayor que el mínimo
            mutable bits = [];                                  // O(1)
            for indexBit in 1..BitSizeI(max) {                  // O(n)
                // Retornar el valor de bits necesarios                                      
                set bits = bits + [ramdomNumberGenerator()];    // O(1)
            }
            for indexBit in 1..BitSizeI(min) {                  // O(n)
                set bits = bits + [ramdomNumberGenerator()];    // O(1)
            }
            set output = ResultArrayAsInt(bits);                // O(1)
        } until (output >= min and output <= max );             // O(1)
        Message($"Número aleatorio entre {min} y {max}: ");     // O(1)
        return output;                                          // O(1)
    }

randomNumberRange

In [6]:
%simulate randomNumberRange min=10 max=20

Número aleatorio entre 10 y 20: 


18

In [7]:
%simulate randomNumberRange min=100 max=200

Número aleatorio entre 100 y 200: 


127

In [8]:
%simulate randomNumberRange min=1000 max=2000 

Número aleatorio entre 1000 y 2000: 


1126

In [9]:
%simulate randomNumberRange min=9999 max=19999

Número aleatorio entre 9999 y 19999: 


19739

## Algoritmo de Euclides
1. Hallar el máximo común divisor para identificar los factores primos.
    + Si restamos el número menor del número mayor, el máximo común divisor no cambia.
    + Mantener restas sucesivas al número mayor.
    + Dividir el número menor
        + Detenerse cuando el residuo es 0.

## Método 1: Llamada directa a la función

In [6]:
function euclideanAlgoGCDm1(a: Int, N: Int) : Int {
                                                        // Complejidad algorítmica: O(2) = O(1)
        Message($"GCD de {a} y {N} es...");             // O(1)
        return GreatestCommonDivisorI((a),(N));         // O(1)
}

euclideanAlgoGCDm1

In [11]:
%simulate euclideanAlgoGCDm1 a=60 N=48

GCD de 60 y 48 es...


12

## Método 2: Versión recursiva

In [7]:
operation euclideanAlgoGCDm2(a: Int, N: Int) : Int {
                                                        // Complejidad algorítmica: O(3) = O(1), O(log(n))
        if((N) == 0) {                                  // O(1)
            return a;                                   // O(1)
        } Message($"GCD: {a}");                         // O(1)
        return euclideanAlgoGCDm2((N), (a) % (N));      // O(log(n))
    }

euclideanAlgoGCDm2

In [13]:
%simulate euclideanAlgoGCDm2 a=60 N=48

GCD: 60
GCD: 48


12

## Método 3: Restas sucesivas

In [8]:
operation euclideanAlgoGCDm3(a: Int, N: Int) : Int {
                                                        // Complejidad algorítmica: 8(n), O(log(n)) + O(log(n)) = O(log(n))
        if((a) == 0) {                                  // O(1)
            return (N);                                 // O(1)
        } if ((N) == 0) {                               // O(1)
            return (a);                                 // O(1)
        } if ((a) == (N)) {                             // O(1)
            return (a);                                 // O(1)
        } if ((a) > (N)) {                              // O(1)
            Message($"GCD de {a} y {N}:");              // O(1)
            Message($"Nuevo intento.");
            return euclideanAlgoGCDm3((a)-(N), (N));    // O(log(n))
        } 
        return euclideanAlgoGCDm3((a), (N)-(a));        // O(log(n))
    }

euclideanAlgoGCDm3

In [15]:
%simulate euclideanAlgoGCDm3 a=60 N=48

GCD de 60 y 48:
Nuevo intento.


12

# Implementación del Algoritmo de Shor
## Fase 2: Parte clásica 
1. Determinar si el orden es par
    + gcd=(a,n)=1
        + a^(φ(n)) ≡ 1 (mod n)
2. Una vez realizada la condición del teorema de Euler aplicamos el teorema de euclides.
3. El valor del orden se le resta 1 y se aplica junto con el número primo para encontrar el primer factor.
4. El valor del orden se le suma 1 y se aplica junto con el número primo para encontrar el primer factor.
    + Retorna el factor y el módulo dividido entre el factor.
5. Fin

φ(n): Es equivalente al orden dividido entre dos.

In [9]:
operation possibleFactorsR (module: Int, randomGenerator: Int, r:Int) : (Bool, (Int, Int)) {
                                                                                // Complejidad algorítmica: O(9) = O(1), O(log(n)) + O(log(n)) + O(log(n)) =  O(log(n))
        if r % 2 == 0 {                                                         // O(1)
            let halfExponentiation = ExpModI(randomGenerator, r/2, module);     // O(log(n))
            if halfExponentiation != module - 1 {                               // O(1)
                let factor = MaxI (                                             // O(1)
                    euclideanAlgoGCDm3(halfExponentiation - 1, module),         // O(log(n))
                    euclideanAlgoGCDm3(halfExponentiation + 1, module)          // O(log(n))
                );
                return (true, (factor, module / factor));                       // O(1)
            } else {                                                            // O(1)
                return (false, (1,1));                                          // O(1)
            }
        } else {                                                                // O(1)
        return(false, (1,1));                                                   // O(1)
        } 
    }

possibleFactorsR

## Multiplicador de un entero modular
### Fase 3: Parte cuántica
1. Realiza la multiplicación modular por una constante entera en un registro qubit.
2. Dado un estado de un Qubi, se multiplica una constante y por el modulo de N.
3. Entonces la operación implementa una operación unitaria definida por:

|y⟩↦|(a⋅y) mod N⟩

4. Ejecutar el algoritmo de estimación de fase cuántica para el calculo entre el registro de almacenamiento y computacional.
5. Fin

In [10]:
operation applyFindingOracle (randomGenerator : Int, module: Int, power : Int, target: Qubit[] ) : Unit is Adj + Ctl{
                                                                                                                        // Complejidad algorítmica: O(1), O(log²(n))
        Fact(IsCoprimeI(randomGenerator,module),"El número aleatorio es `randomGenerator`, y el modulo es `module`.");  // O(1)
        // Realización de una multiplicación modular por una constante entera en un registro qubit.
        MultiplyByModularInteger(ExpModI(randomGenerator, power, module),module, LittleEndian(target));                 // O(log²(n))
    }

applyFindingOracle

# Función convergente de fracción continua
Encuentra la fracción continua convergente más cercana a la fracción con el denominador menor o igual al limite del denominador, además, se implementa el calculo del máximo común divisor.


In [11]:
operation periodFrequency (module : Int, frequencyEstimation : Int, bitsPrecision: Int, currentDivisor: Int) : Int {    
                                                                                                                    // Complejidad algorítmica: O(4) = O(1), O(log(n)) + O(1/(log(n))) = O(log(n)) 
        let (s, r) = (ContinuedFractionConvergentI(Fraction(frequencyEstimation, 2 ^ bitsPrecision), module))!;     // O(log(n)) 
        let (sAbsolute, rAbsolute) = (AbsI(s), AbsI(r));                                                            // O(2)
        //  Calcula el máximo común divisor de dos enteros (se llama al método 3 de implementación del GCD).
        return (rAbsolute * currentDivisor) / euclideanAlgoGCDm3(currentDivisor, rAbsolute);                        // O(1/(log(n)))
    } 

periodFrequency

## Estimación de la frecuencia
Esta función realiza la multiplicación de un entero modular a partir de la sumatoria del registro computacional más el registro de almacenamiento desde 0 hasta n bits.

1. Se determina los bits de precisión
2. Se crea una cadena de bits cuánticos de valores propios correspondientes a un vector.
3. Se crea una variable de registro para la codificación de valores con índice 0 (el bit más bajo de un entero sin signo).
4. Aplicación de la operación XOR al registro de los valores propios basados en el entero 1.

|y⟩→|y⊕a⟩

5. Se recorre el indicé de los bits de precisión
    + Dentro de cada valor con un bit cuántico en superposición, se alplica una rotación sobre el estado |1⟩.

R1(n,k):=diag(1,e^(iπk/2n))

6. Si el valor de la medición se encuentra en la base Z es igual a 1
    + Se reestablece al estado inicial después de la medición
    + Se asigna la frecuencia de estimación
7. Se resetean los valores propio.
8. se retorna el valor de la frecuencia.
9. Fin

  
    

In [12]:
operation EstimateFrequencyValue (randomGenerator : Int, module: Int, bits : Int) : Int { 
                                                                                            // Complejidad algorítmica: O(12) = O(1), O(n) + O(n) + O(log²(n)) = O(log²(n))
        mutable frequencyEstimation = 0;                                                    // O(1)
        
        let bitsPrecision = 2 * bits + 1;                                                   // O(1)

        use eigenStateRegister = Qubit[bits];                                               // O(1)
        
        let eigenStateRegisterLittleEndian = LittleEndian(eigenStateRegister);              // O(1)

        // Aplicando una operación XOR entre un entero clásico y un entero representado por un registro de qubits.
        ApplyXorInPlace(1, eigenStateRegisterLittleEndian);                                 // O(n) // Suma de n registros
        let oracle = applyFindingOracle(randomGenerator, module, _, _);                     // O(log²(n))

        use a = Qubit();                                                                    // O(1)

        for index in bitsPrecision -1..-1..0 {                                              // O(n)
            within {
                H(a);                                                                       // O(1)
            } apply {
                Controlled oracle([a], (1 <<< index, eigenStateRegisterLittleEndian!));     // O(1)      
                // Aplicar una rotación sobre el estado |1> por un ángulo especificado como una fracción diádica.
                R1Frac(frequencyEstimation, bitsPrecision - 1 - index, a);                  // O(1)
            } if MResetZ(a) == One {                                                        // O(1)
                set frequencyEstimation += 1 <<< (bitsPrecision - 1 - index);               // O(1)
            }
        }
        // Mide los qubits y se asegura de que estén en el estado |0⟩ para que puedan liberarse de forma segura.
        ResetAll(eigenStateRegister);                                                       // O(1)
        return frequencyEstimation;                                                         // O(1)
    }

EstimateFrequencyValue

## Algoritmo de estimación de fase
En este apartado se identifican los eigenvalues de operadores unitarios, dado un operador unitario U y un estado |ϕ⟩ tal que |ϕ⟩ es un autoestado de U con un eigenvalue desconocido ϕ, a este problema se le conoce como estimación de fase.

U|ϕ⟩=ϕ|ϕ⟩

1. Revisar si una condición clásica es verdadera.
    + Se revisa si el número aleatorio y el módulo son coprimos entre sí.
2. Se almacena el resultado de la estimación de fase dentro de una variable muteable.
3. Si la estimación de frecuencia es diferente de 0, entonces se asigna el valor del periodo con los valores determinados.
4. Si la estimación de frecuencia es igual a 0, se notifica al usuario.
5. Se retorna el valor.
6. Fin


 

In [13]:
operation phaseEstimation (randomGenerator: Int, module: Int ) : Int {
                                                                                                                        // Complejidad algorítmica: O(9) = O(1), O(n) +  O(log(n)) + O(log²(n)) = O(log²(n)) 
        Fact(IsCoprimeI(randomGenerator,module),"El numero aleatorio es `randomGenerator` y el modulo es `module`.");   // O(1)

        mutable result = 1;                                                                                             // O(1)

        let bits = BitSizeI(module);                                                                                    // O(n) // a < 2^n     

        let bitsPrecision = 2 * bits + 1;                                                                               // O(1)

        mutable frequencyEstimation = 0;                                                                                // O(1)

        set frequencyEstimation = EstimateFrequencyValue(randomGenerator, module,bits);                                 // O(log²(n))
        
        if  frequencyEstimation != 0 {                                                                                  // O(1)
            set result = periodFrequency (module, frequencyEstimation, bitsPrecision, result);                          // O(log(n))
        } else {                                                                                                        // O(1)
            Message("La frecuencia estimada tiene el valor 0.");                                                        // O(1)
        }
        return result;                                                                                                  // O(1)
    }

phaseEstimation

## Prueba de implementación del algoritmo Shor
1. Se determina si el número es par
    + Si el número es par, se calcula n/2
2. Se establecen los factores predeterminados
3. Hasta que los valores de los factores esten establecidos, verificar si el número aleatorio y el número primo son coprimos:
    + Calcular el orden por medio de la estimación de fase.
    + Asignar los valores por defecto a los posibles factores encontrados.
4. Si no son coprimos, ejecutar el algoritmo de euclides directamente.
5. Si la condición es diferente, se bloquea y continua la iteración.
6. Se retornan los factores.
7. Fin


In [14]:
operation shorImplementatonTest (n: Int): (Int, Int) {
                                                                                                // Complejidad algorítmica: O(16) = O(1)+O(n)[log²(n)+ log(n)+log(n)]=O(nlog²(n))
        if n % 2 == 0 {                                                                         // O(1)
            // Comprobar si hay un número par.
            Message("Caso trivial: El número es par.");                                         // O(1)
            return (n/2, 2);                                                                    // O(1)
        }
        
        // Configura los factores desconocidos y establece los valores predeterminados.
        mutable setUpFactors = false;                                                           // O(1)
        mutable defaultFactors = (1,1);                                                         // O(2)

    
        repeat {                                                                                // O(n)
            let randomGenerator = randomNumberRange(2, n - 1);                                  // O(1)
            // DrawRandomInt 
            // Casos:  IBM(2, N-1),  MICROSOT(1 < a < N-1)
            if IsCoprimeI (randomGenerator,n) {                                                 // O(1)
                Message($"Número aleatorio: {randomGenerator}");                                // O(1)
                let r = phaseEstimation(randomGenerator, n);                                    // O(log²(n))
                set (setUpFactors,defaultFactors) = possibleFactorsR(n, randomGenerator, r);    // O(log(n))
            }  else {                                                                           // O(1)
                let GCD = euclideanAlgoGCDm3(n,randomGenerator);                                // O(log(n))
                Message($"Divisor optenido: {n}, GCD: {GCD}.");                                 // O(1)
                set setUpFactors = true;                                                        // O(1)
                set defaultFactors = (GCD, n/ GCD);                                             // O(1)
            }
        } until setUpFactors                                                                    // O(1)
        fixup {                                                                                 // O(1)
        Message("La estimación del periodo no retorna un factor valido.");                      // O(1)
        }
        return defaultFactors;                                                                  // O(1)
    }

shorImplementatonTest

In [22]:
%simulate shorImplementatonTest n=15

Número aleatorio entre 2 y 14: 
GCD de 15 y 6:
Nuevo intento.
GCD de 9 y 6:
Nuevo intento.
Divisor optenido: 15, GCD: 3.


(3, 5)

In [29]:
%simulate shorImplementatonTest n=15

Número aleatorio entre 2 y 14: 
Número aleatorio: 11
GCD de 10 y 5:
Nuevo intento.
GCD de 12 y 3:
Nuevo intento.
GCD de 9 y 3:
Nuevo intento.
GCD de 6 y 3:
Nuevo intento.


(5, 3)

In [30]:
%simulate shorImplementatonTest n=15

Número aleatorio entre 2 y 14: 
GCD de 15 y 3:
Nuevo intento.
GCD de 12 y 3:
Nuevo intento.
GCD de 9 y 3:
Nuevo intento.
GCD de 6 y 3:
Nuevo intento.
Divisor optenido: 15, GCD: 3.


(3, 5)

In [33]:
%simulate shorImplementatonTest n=21

Número aleatorio entre 2 y 20: 
GCD de 21 y 3:
Nuevo intento.
GCD de 18 y 3:
Nuevo intento.
GCD de 15 y 3:
Nuevo intento.
GCD de 12 y 3:
Nuevo intento.
GCD de 9 y 3:
Nuevo intento.
GCD de 6 y 3:
Nuevo intento.
Divisor optenido: 21, GCD: 3.


(3, 7)

In [38]:
%simulate shorImplementatonTest n=21

Número aleatorio entre 2 y 20: 
GCD de 21 y 6:
Nuevo intento.
GCD de 15 y 6:
Nuevo intento.
GCD de 9 y 6:
Nuevo intento.
Divisor optenido: 21, GCD: 3.


(3, 7)

In [39]:
%simulate shorImplementatonTest n=21

Número aleatorio entre 2 y 20: 
Número aleatorio: 5
GCD de 4 y 1:
Nuevo intento.
GCD de 3 y 1:
Nuevo intento.
GCD de 2 y 1:
Nuevo intento.
GCD de 6 y 3:
Nuevo intento.


(3, 7)

In [40]:
%simulate shorImplementatonTest n=95

Número aleatorio entre 2 y 94: 
GCD de 95 y 55:
Nuevo intento.
GCD de 40 y 15:
Nuevo intento.
GCD de 25 y 15:
Nuevo intento.
GCD de 10 y 5:
Nuevo intento.
Divisor optenido: 95, GCD: 5.


(5, 19)

In [41]:
%simulate shorImplementatonTest n=95

Número aleatorio entre 2 y 94: 
Número aleatorio: 62
La estimación del periodo no retorna un factor valido.
Número aleatorio entre 2 y 94: 
Número aleatorio: 66
La estimación del periodo no retorna un factor valido.
Número aleatorio entre 2 y 94: 
Número aleatorio: 63
GCD de 38 y 19:
Nuevo intento.
GCD de 40 y 15:
Nuevo intento.
GCD de 25 y 15:
Nuevo intento.
GCD de 10 y 5:
Nuevo intento.


(19, 5)

In [42]:
%simulate shorImplementatonTest n=95

Número aleatorio entre 2 y 94: 
Número aleatorio: 27
GCD de 38 y 19:
Nuevo intento.
GCD de 40 y 15:
Nuevo intento.
GCD de 25 y 15:
Nuevo intento.
GCD de 10 y 5:
Nuevo intento.


(19, 5)

In [15]:
%simulate shorImplementatonTest n=958

Caso trivial: El número es par.


(479, 2)

In [16]:
%simulate shorImplementatonTest n=958

Caso trivial: El número es par.


(479, 2)

In [17]:
%simulate shorImplementatonTest n=998

Caso trivial: El número es par.


(499, 2)

In [18]:
%simulate shorImplementatonTest n=1059

Número aleatorio entre 2 y 1058: 
Número aleatorio: 50


## Conclusiones
1. Según el código previo, la complejidad computacional temporal es O(nlog²(n)).
Esto debido a las distintas repeticiones a evaluar en un número aleatorio.
2. Según el código previo, la complejidad computacional espacial es O(log²(n)).
Esto debido a la cantidad de veces que recorre la celda de la multiplicación modular.